In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


### Read Data

In [2]:
bank_rate = pd.read_csv('./Data/BoE-Database_export.csv')
fed_funds = pd.read_csv('./Data/FEDFUNDS.csv')
ftse100 = pd.read_csv('./Data/FTSE 100 Historical Price Data.csv')
sp500 = pd.read_csv('./Data/S&P 500 Historical Price Data.csv')
us_inflation = pd.read_csv('./Data/US Inflation RAte.csv')
uk_inflation = pd.read_csv('./Data/series-290524.csv')

In [3]:
bank_rate['Date'] = pd.to_datetime(bank_rate['Date'], format='%d-%m-%Y')
fed_funds['observation_date'] = pd.to_datetime(fed_funds['observation_date'])
ftse100['Date'] = pd.to_datetime(ftse100['Date'])
sp500['Date'] = pd.to_datetime(sp500['Date'])
us_inflation['Date'] = pd.to_datetime(us_inflation['Date'], format='%b-%y')
uk_inflation['Date'] = pd.to_datetime(uk_inflation['Date'], format='%b-%y')



/var/folders/tl/gr7ky_ln79195ycwht64zv_r0000gn/T/ipykernel_10215/1995424870.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ftse100['Date'] = pd.to_datetime(ftse100['Date'])
/var/folders/tl/gr7ky_ln79195ycwht64zv_r0000gn/T/ipykernel_10215/1995424870.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  sp500['Date'] = pd.to_datetime(sp500['Date'])


### Merge Data

In [4]:
# Convert all to monthly data if necessary, using the average or last value
sp500 = sp500.set_index('Date').resample('M').last()
fed_funds = fed_funds.set_index('observation_date').resample('M').last()
us_inflation = us_inflation.set_index('Date').resample('M').last()

# Merge datasets on the date index
df = pd.concat([sp500[['Price']], fed_funds['FEDFUNDS'], us_inflation['Inflation Rates']], axis=1)
df.columns = ['SP500_Price', 'FEDFUNDS', 'US_Inflation']


### Data Cleaning

In [5]:
df['SP500_Price'] = df['SP500_Price'].replace('[\$,]', '', regex=True)
df['SP500_Price'] = pd.to_numeric(df['SP500_Price'], errors='coerce')
df.dropna(inplace=True)



### Regression Analysis

In [6]:
X = df[['FEDFUNDS', 'US_Inflation']] # 
X = sm.add_constant(X)  # Adds a constant term to the predictor
y = df['SP500_Price']

In [7]:
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            SP500_Price   R-squared:                       0.465
Model:                            OLS   Adj. R-squared:                  0.451
Method:                 Least Squares   F-statistic:                     35.13
Date:                Fri, 31 May 2024   Prob (F-statistic):           1.03e-11
Time:                        10:02:35   Log-Likelihood:                -649.78
No. Observations:                  84   AIC:                             1306.
Df Residuals:                      81   BIC:                             1313.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         2576.2526    124.063     20.766   